In [0]:
import io, os
import re as re
import zipfile as zipfile
import math
import numpy as np
import random
from operator import itemgetter
import itertools
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import scipy.spatial.distance
from sklearn.metrics import accuracy_score
import random

In [224]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
textsDocNumbers = {}       # [ doc: text, ... ]
authorsTextNumbers = {}         # to split train and test data

mytextzip = ''
path = '/content/drive/My Drive/30Columnists.zip'

with zipfile.ZipFile(path) as z:
    for zipinfo in z.infolist():
        if zipinfo.filename.endswith('.txt') and re.search('raw_texts', zipinfo.filename):
            with z.open(zipinfo) as f:
                textfile = io.TextIOWrapper(f, encoding='cp1254', newline='')
                for line in textfile:
                    if len(line.strip()): mytextzip += line.strip()

                filename = str(zipinfo.filename)
                filename = filename.split("/")
                filename = filename[-1][:-4]

                mytextzip = mytextzip.lower()
                
                textsDocNumbers[filename] = mytextzip

                author, doc = filename.split(".")
                if author not in authorsTextNumbers:
                    authorsTextNumbers[author] = 1
                else:
                    authorsTextNumbers[author] += 1

                mytextzip = ''

In [226]:
for doc, text in textsDocNumbers.items():
    print("key: " + doc + "\nvalue: " + text)
    break

key: 1.1
value: there can be no doubt that edinburgh is a huge tourist attraction both within the united kingdom and internationally. there is so much to see in scotland, but its capital is arguably its biggest gateway if not its biggest draw.capitals are not always that lucky. i have never met anyone who has, by choice, travelled to australia to visit canberra  or to peru to see lima.although it can sometimes be difficult on a driech march morning when the wind is howling up your trouser legs and the drizzle is misting up your spectacles, edinburgh is an often idyllic, romantic and fun-packed destination.i remember being amazed a couple of decades ago when some english friends of mine told me they were coming to edinburgh on their honeymoon  i had never appreciated my home as a place for newlyweds to choose for their first married tryst. now edinburgh is in the multi-million pound premier league for british honeymoons  and why not?tourism has become a hugely important sector of the ed

In [227]:
print(authorsTextNumbers)

{'1': 50, '10': 50, '11': 50, '12': 50, '13': 50, '14': 50, '15': 50, '16': 50, '17': 50, '18': 50, '19': 50, '2': 50, '20': 50, '21': 50, '22': 50, '23': 50, '24': 50, '25': 50, '26': 50, '27': 50, '28': 50, '29': 50, '3': 50, '30': 50, '4': 50, '5': 50, '6': 50, '7': 50, '8': 50, '9': 50}


In [0]:
def splitTrainTest(dict, n):
    train = {}      # { author: [text1's doc number, text2's doc number,...], ... }
    test_a = []     # [ 1. text's author, ... ]
    test_t = []     # [ 1. text's doc number, ... ]

    temp = {}       # to keep random index to selecting which texts will be test text
    for aut, number in authorsTextNumbers.items():
        temp[aut] = []
        while len(temp[aut]) < n:
            x = random.randint(0, number-1)
            if x not in temp[aut]:
                temp[aut].append(x)
        temp[aut] = sorted(temp[aut])

    # temp = {'1': [16, 28, 29, 39, 47], '10': [0, 6, 19, 23, 24], ...}

    for doc in dict.keys():
        # doc number --> author, text number
        author = doc.split(".")
        author = author[0]

        if author not in train:
            train[author] = []

        if len(train[author]) in temp[author]:
            temp[author].remove(len(train[author]))
            for j, num in enumerate(temp[author]):
                temp[author][j] -= 1
            test_t.append(doc)
            test_a.append(author)   # to prediction
            
        else:
            train[author].append(doc)

    return train, test_a, test_t

In [0]:
train, test_a, test_t = splitTrainTest(textsDocNumbers, 5)

In [230]:
print("len(train) =",len(train))
print("len(train['1']) =",len(train["1"]))
print("len(train['10']) =",len(train["10"]))

print("len(test_a) =",len(test_a))
print("len(test_t) =",len(test_t))

len(train) = 30
len(train['1']) = 45
len(train['10']) = 45
len(test_a) = 150
len(test_t) = 150


In [0]:
def splitIntoWords(text):
    text = text.strip()

    text = text.replace("(","")
    text = text.replace(")","")

    text = text.replace("”",'"')
    text = text.replace(',"','"')
    text = text.replace(",", "")
    text = text.replace("\n", ".")
    text = text.replace("  ", " ")

    text = re.split(r'\W', text.lower())

    return text

In [0]:
def createDB(aut, docs, DB, n):

    temp = []
    for doc in docs:
        text = textsDocNumbers[doc]
        temp.append(splitIntoWords(text))

    # DB = { author1: { word1: [ 'word1's count in author1's texts , {word2: 'word1 word2' pairs' count in author1's texts, ...} ], ... }, ...}
    # DB = { "1": { "this": [5, {"is": 2, ...} ], ... }, "2": {}, ...}

    DB[aut] = {}

    for text in temp:
        for i, word in enumerate(text):
            if i == len(text)-n:
                break

            else:
                for j in range(n-1):
                    if j == 0:
                        continue
                    word += " " + text[i+j]

                if word not in DB[aut]:
                    DB[aut][word] = []
                    DB[aut][word].append(0)
                    DB[aut][word].append({})
                
                after = text[i+n-1]         # next word
                DB[aut][word][0] += 1       # word1's count in text

                # DB[aut][word][1]  dictionary contains words that come after word1 as key and their counts as value

                if after not in DB[aut][word][1]:
                    DB[aut][word][1][after] = 1
                else:
                    DB[aut][word][1][after] += 1

    return DB

In [233]:
# Create DB

n = int(input('Enter n: '))
DB = {}
for aut, docs in train.items():
    DB = createDB(aut, docs, DB, n)

# n = 2
# print(DB["1"])
# {'there': [150, {'can': 5, 'is': 21, 'and': 1,...}], 'can': [132, {'be': 20, 'sometimes': 1, 'pull': 1, 't': 19,...], ...}

# n = 3
# print(DB["1"])
# {'there can': [5, {'be': 4, 'never': 1}], 'can be': [20, {'no': 3, 'done': 2,...}], ...}

# n = 4
# print(DB["1"])
# {'there can be': [4, {'no': 3, 'little': 1}], 'can be no': [3, {'doubt': 2, 'doubting': 1}], ...}

Enter n: 2


In [234]:
if n == 2:
    print("The number of 'united' in 1's texts:\n", DB["1"]["united"][0])
    print("The number of 'united kingdom' in 1's texts:\n", DB["1"]["united"][1]["kingdom"])
    print("The words and their numbers after 'united' in 1's texts:\n", DB["1"]["united"][1])

The number of 'united' in 1's texts:
 12
The number of 'united kingdom' in 1's texts:
 5
The words and their numbers after 'united' in 1's texts:
 {'kingdom': 5, 'states': 5, 'nations': 1, 'the': 1}


In [0]:
predictions = []

for doc in test_t:
    pr = {}     # to find best author prediction for text
                # { author: prediction value, ... }  

    txt = textsDocNumbers[doc]
    words = splitIntoWords(txt)

    for aut in train.keys():

        pr[aut] = 0

        for i, word in enumerate(words):
            if i == len(words)-n:
                break

            for j in range(n-1):
                if j == 0:
                    continue
                word += " " + words[i+j]

            after = words[i+n-1]      # next word

            if word in DB[aut]:
                cTotal = DB[aut][word][0]               # of "word" in train text
                if after in DB[aut][word][1]:
                    cPart = DB[aut][word][1][after]     # of "word after" in train text

                    p = cPart / cTotal      # prediction
                else:
                    p = 1e-10
            else:
                p = 1e-10

            # to prevent errors;
            # if one of words(word or after) not in author's texts,
            # p will equal very small number

            pr[aut] += math.log(p)

    # to find the best author prediction
    pr = sorted(pr.items(), key=lambda pr: pr[1], reverse=True)
    # pr[0][0] --> the best author
    predictions.append(pr[0][0])

In [236]:
print(predictions)

['1', '1', '1', '1', '1', '10', '10', '10', '10', '10', '9', '4', '11', '16', '16', '12', '12', '12', '12', '12', '13', '13', '13', '13', '13', '12', '14', '14', '14', '12', '15', '15', '12', '15', '15', '16', '16', '16', '16', '16', '17', '17', '17', '17', '17', '19', '19', '18', '18', '18', '19', '19', '19', '19', '19', '2', '2', '2', '2', '2', '20', '20', '20', '20', '20', '21', '21', '21', '21', '21', '22', '22', '22', '22', '22', '23', '23', '23', '23', '23', '24', '24', '24', '24', '24', '25', '25', '25', '25', '25', '26', '26', '26', '26', '26', '27', '27', '27', '27', '16', '12', '8', '12', '12', '28', '16', '10', '16', '16', '16', '3', '3', '10', '3', '3', '30', '17', '10', '30', '10', '4', '4', '4', '4', '4', '5', '5', '5', '5', '10', '10', '10', '6', '10', '10', '10', '10', '10', '10', '10', '12', '12', '8', '8', '8', '10', '11', '11', '9', '9']


In [237]:
acc = 0
for i in range(len(predictions)):
    if predictions[i] == test_a[i]:
        acc += 1

print(acc, "/", len(predictions))
print("accuracy:", (acc / len(predictions)))

# n = 2
# 105 / 150     ---> accuracy: 0.7
# 113 / 150     ---> accuracy: 0.75 
# 111 / 150     ---> accuracy: 0.74

# n = 3
# 111 / 150     ---> accuracy: 0.74

# n = 4
# 107 / 150     ---> accuracy: 0.71

112 / 150
accuracy: 0.7466666666666667


In [0]:
import gc
import sys

def get_obj_size(obj):
    marked = {id(obj)}
    obj_q = [obj]
    sz = 0

    while obj_q:
        sz += sum(map(sys.getsizeof, obj_q))

        # Lookup all the object referred to by the object in obj_q.
        # See: https://docs.python.org/3.7/library/gc.html#gc.get_referents
        all_refr = ((id(o), o) for o in gc.get_referents(*obj_q))

        # Filter object that are already marked.
        # Using dict notation will prevent repeated objects.
        new_refr = {o_id: o for o_id, o in all_refr if o_id not in marked and not isinstance(o, type)}

        # The new obj_q will be the ones that were not marked,
        # and we will update marked with their ids so we will
        # not traverse them again.
        obj_q = new_refr.values()
        marked.update(new_refr.keys())

    return sz

# get_obj_size(DB)

# n = 2
# 71962948 = 71.96

# n = 3
# 252950532 = 252.95